# End of week 1 exercise

A simple assistnat to answer any question with details given a question and question type (defaulting to `code` for now)

Features:
- Receives question and question type
- Streams the responses to a question
- Has both ollama with Llama3.2 and OpenAi with gpt-4o-mini implementations

In [18]:
# imports
import sys
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

base_path = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(base_path)



In [20]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
ollama_base_url = "http://localhost:11434"
openrouter_base_url = "https://openrouter.ai/api/v1"

In [ ]:
#verify ollama is running 
requests.get("http://localhost:11434").content
# Let's pull the llama3.2 model
!ollama pull llama3.2

In [25]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

if api_key and api_key.startswith('sk-or-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI(base_url=openrouter_base_url, api_key=api_key)
ollama_openai = OpenAI(base_url=ollama_base_url, api_key="ollama")

API key looks good so far


In [15]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [13]:
def get_system_prompt(question_type='code'):
    system_prompt = f"""
    You are provided with a question of type {question_type}. You are an expert in the field of {question_type} and you are able to answer questions about the {question_type}.
    When answering, be concise and to the point.
    Include information such as:
    - The purpose of the {question_type}
    - Any other relevant information

    Format your response in markdown to be arranged as:
     - Title
     - Body
     - Conclusion
     - Recommendations
    """
    return system_prompt


In [6]:
def get_user_prompt(question, question_type='code'):
    user_prompt = f"""
    You are analysing the given {question} provided of type {question_type}. 
    The goal is to provide a concise and to the point answer to the question.
    Include all relevant information including:
    - The purpose of the {question}
    - Any other relevant information
    """
    return user_prompt


In [7]:
def messages_for_gpt(question, question_type='code'):
    system_prompt = get_system_prompt(question_type)
    user_prompt = get_user_prompt(question, question_type)
    return [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]


In [8]:
# Get gpt-4o-mini to answer, with streaming
def stream_gpt_response(question, question_type='code'):
    messages = messages_for_gpt(question, question_type)

    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
# Get Llama 3.2 to answer
def stream_llama_response(question, question_type='code'):
    messages = messages_for_gpt(question, question_type)
    response = ollama_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages)
    display(Markdown(response.choices[0].message.content))


In [ ]:
# stream openai response
stream_gpt_response(question)

In [33]:
#stream llama response
stream_llama_response(question)

NotFoundError: 404 page not found

In [ ]:
# now let's see how it works with a different question
stream_gpt_response("What's your take on the current state of the world?", "general")
